# Running BOA Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib
import shutil

from IPython.display import Code
from wrapper import Wrapper

import boa

[WARNING 07-12 15:40:39] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("boa_runs*")
for path in old_runs:
    shutil.rmtree(path)

## Loading the Config File

In [3]:
config_path = pathlib.Path().resolve() / "single_config.yaml"

Here we can see what the configuration file looks like

In [4]:
Code(config_path)

# Single objective optimization config
optimization_options:
    objective_options:
        objectives:
            - name: Cosine8
    trials: 50
    append_timestamp: False
parameters:
    x0:
        type: range
        bounds: [0.0, 1.0]
    x1:
        type: range
        bounds: [0.0, 1.0]
    x2:
        type: range
        bounds: [0.0, 1.0]
    x3:
        type: range
        bounds: [0.0, 1.0]
    x4:
        type: range
        bounds: [0.0, 1.0]
    x5:
        type: range
        bounds: [0.0, 1.0]
    x6:
        type: range
        bounds: [0.0, 1.0]
    x7:
        type: range
        bounds: [0.0, 1.0]
# These are all defaults, so we don't need to specify them in this case
#script_options:
#    wrapper_path: ./wrapper.py
#    wrapper_name: Wrapper
#    working_dir: .
#    experiment_dir: ... # this is where boa will write logs to by default
                         # if not specified it will be working_dir/experiment_name
#    append_timestamp: True
# This last option appends a timestamp to our output experiment directory.
# This is also the default (True)

we need the config normalized, which modifies the parameter section
into a less user friendly form, but what the downstream libraries need

In [5]:
config = boa.load_jsonlike(config_path)

## Define Our Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [6]:
Code(Wrapper.path())

import numpy as np
from ax.utils.measurement.synthetic_functions import from_botorch
from botorch.test_functions.synthetic import Cosine8

import boa

cosine8 = from_botorch(Cosine8())


def black_box_model(X) -> float:
    result = -cosine8(X)
    return result


class Wrapper(boa.BaseWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = {}

    def run_model(self, trial) -> None:
        X = np.array([parameter for parameter in trial.arm.parameters.values()])
        # This is a silly toy function, in reality,
        # you could instead import your model main() function and use that, and then collect the results.
        # You could also call an external script to start a model run from Bash or elsewhere.
        self.data[trial.index] = black_box_model(X)

    def set_trial_status(self, trial) -> None:
        data_exists = self.data.get(trial.index)
        if data_exists:
            trial.mark_completed()

    def fetch_trial_data(self, trial, *args, **kwargs):
        return self.data[trial.index]

## Initialize our Setup

In [7]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-12 15:40:41] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x0. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-12 15:40:41] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-12 15:40:41] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-12 15:40:41] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-12 

(Scheduler(experiment=Experiment(boa_runs), generation_strategy=GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 16 trials, GPEI for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <wrapper.Wrapper at 0x15ab0ee60>)

## Run our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [8]:
scheduler = controller.run()

[INFO 2023-07-12 15:40:41,021 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/boa_runs_20230712T154041
Start Time: 20230712T154041
Version: 0.8.7.dev0+g78bf53c.d20230712

##############################################

[INFO 07-12 15:40:41] Scheduler: Running trials [0]...
[INFO 07-12 15:40:41] Scheduler: Running trials [1]...
[INFO 07-12 15:40:42] Scheduler: Running trials [2]...
[INFO 07-12 15:40:43] Scheduler: Running trials [3]...
[INFO 07-12 15:40:44] Scheduler: Running trials [4]...
[INFO 07-12 15:40:45] Scheduler: Running trials [5]...
[INFO 07-12 15:40:46] Scheduler: Running trials [6]...
[INFO 07-12 15:40:47] Scheduler: Running trials [7]...
[INFO 07-12 15:40:48] Scheduler: Running trials [8]...
[INFO 07-12 15:40:49] Scheduler: Running trials [9]...
[INFO 07-12 15:40:50] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-12 15:40:50] Scheduler: F

## Get the Best Trial and Output All Trials

`best_fitted_trials` uses the data to do a fitting from all trials and with the noise levels you provided (or if no noise levels was provided, it assumed an unknown level of noise and inferred the noise level from the trial runs)

In [9]:
trial = scheduler.best_fitted_trials()
trial

{49: {'params': {'x0': 0.4029266914532883,
   'x1': 0.39405910818785606,
   'x2': 0.0,
   'x3': 0.0,
   'x4': 0.056054701425135314,
   'x5': 0.3000685003580652,
   'x6': 0.33318118192182516,
   'x7': 0.0},
  'means': {'Cosine8': -0.09115523653444302},
  'cov_matrix': {'Cosine8': {'Cosine8': 1.7854182811629313e-06}}}}

if you need the exact points of the best trial, maybe because you need the trial number of the best trial to plot results, or for any other reason, `best_raw_trails` does not do any fitting

In [10]:
trial = scheduler.best_raw_trials()
trial

{49: {'params': {'x0': 0.4029266914532883,
   'x1': 0.39405910818785606,
   'x2': 0.0,
   'x3': 0.0,
   'x4': 0.056054701425135314,
   'x5': 0.3000685003580652,
   'x6': 0.33318118192182516,
   'x7': 0.0},
  'means': {'Cosine8': -0.09121045068891717},
  'cov_matrix': {'Cosine8': {'Cosine8': 0.0}}}}

## Output a DataFrame of All Trials

In [11]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,Cosine8,x0,x1,x2,x3,x4,x5,x6,x7
0,0,0_0,COMPLETED,Sobol,4.548841,0.952413,0.698241,0.556016,0.924099,0.948844,0.158136,0.877891,0.088834
1,1,1_0,COMPLETED,Sobol,4.114170,0.389821,0.577049,0.991525,0.203714,0.030380,0.621225,0.948096,0.978112
2,2,2_0,COMPLETED,Sobol,2.855460,0.897384,0.363080,0.412098,0.721599,0.820840,0.109188,0.889895,0.327299
3,3,3_0,COMPLETED,Sobol,3.046155,0.286399,0.523100,0.759998,0.763263,0.141199,0.666532,0.941039,0.170015
4,4,4_0,COMPLETED,Sobol,2.811232,0.726105,0.497294,0.418107,0.560933,0.888328,0.499065,0.764801,0.145881
5,5,5_0,COMPLETED,Sobol,4.146668,0.032876,0.989555,0.749453,0.456220,0.797499,0.819627,0.914055,0.805068
6,6,6_0,COMPLETED,Sobol,2.437903,0.908188,0.791685,0.078245,0.319633,0.622762,0.460645,0.512239,0.278182
7,7,7_0,COMPLETED,Sobol,2.929330,0.878969,0.315119,0.689226,0.935065,0.454900,0.176417,0.552799,0.122317
8,8,8_0,COMPLETED,Sobol,5.557501,0.508197,0.962551,0.311967,0.926938,0.991895,0.905188,0.925331,0.645691
9,9,9_0,COMPLETED,Sobol,3.860062,0.884691,0.814058,0.127347,0.575577,0.993630,0.969971,0.041379,0.103028
